# Automatyczne pozyskiwanie danych

## Tomasz Rodak

Wykład 7

---

## Scrapy

Na podstawie: [Scrapy tutorial](https://docs.scrapy.org/en/latest/intro/tutorial.html)

Scrapy to framework do tworzenia botów sieciowych wykonujących ekstrakcję danych z witryn internetowych. Jest napisany w Pythonie.

### Tworzenie projektu

Projekt to katalog, w którym znajdują się wszystkie pliki konfiguracyjne i kod źródłowy bota. Projekt tworzy się automatycznie poleceniem w terminalu:

```python
scrapy startproject <nazwa_projektu>
```

Przejdź do katalogu, w którym chcesz utworzyć projekt i uruchom polecenie: 

```bash
scrapy startproject tutorial
```

Powstanie wówczas katalog `tutorial` postaci:

```
tutorial/
    scrapy.cfg
    tutorial/
        __init__.py
        items.py
        middlewares.py
        pipelines.py
        settings.py
        spiders/
            __init__.py
```

- `scrapy.cfg` – główny plik konfiguracyjny projektu
- `tutorial/` – katalog z kodem źródłowym bota
    - `__init__.py` – inicjalizuje moduł
    - `items.py` – definicje struktur danych (elementów zbieranych przez bota)
    - `middlewares.py` – definicje middleware (pośredników obsługujących żądania i odpowiedzi)
    - `pipelines.py` – definicje potoków przetwarzania danych
    - `settings.py` – ustawienia projektu (np. limity, nagłówki, user-agent)
    - `spiders/` – katalog z definicjami pająków (klas zbierających dane z witryn)
        - `__init__.py` – inicjalizuje moduł spiders

### Pierwszy pająk

Pająki to klasy dziedziczące po klasie `Spider`, które wykonują ekstrakcję danych z jednej lub wielu witryn internetowych. Muszą one definiować początkowe żądania do wykonania, a opcjonalnie także sposób podążania za linkami i przetwarzania pobranych stron w celu ekstrakcji danych.

Oto kod naszego pierwszego pająka. Jego zadaniem będzie pobieranie stron z cytatami z witryny [quotes.toscrape.com](https://quotes.toscrape.com).
Kod zapisz w pliku `quotes_spider.py` w katalogu `tutorial/spiders`:

```python
from pathlib import Path

import scrapy


class QuotesSpider(scrapy.Spider):
    name = "quotes"

    async def start(self):
        urls = [
            "https://quotes.toscrape.com/page/1/",
            "https://quotes.toscrape.com/page/2/",
        ]
        for url in urls:
            yield scrapy.Request(url=url, callback=self.parse)

    def parse(self, response):
        page = response.url.split("/")[-2]
        filename = f"quotes-{page}.html"
        Path(filename).write_bytes(response.body)
        self.log(f"Saved file {filename}")
```

Atrybuty i metody klasy `QuotesSpider`:
- `name` – unikalna nazwa pająka, używana do identyfikacji go w projekcie.
- `start()` – funkcja generatora zwracająca początkowe żądania (`Request`). Parametr `urls` to lista adresów URL, które pająk ma odwiedzić. Każde żądanie jest tworzone przez `scrapy.Request`, gdzie `url` to adres strony, a `callback` to metoda wywoływana po pobraniu strony.
- `parse()` – metoda wywoływana po pobraniu strony. Odpowiada za przetwarzanie odpowiedzi i ekstrakcję danych. W tym przypadku zapisuje zawartość strony do pliku HTML. Parametr `response` jest instancją klasy `TextResponse`.

Typowo metoda `parse()` odpowiada za przetwarzanie odpowiedzi z serwera – polega to na ekstrakcji interesujących danych (np. do słowników lub obiektów Item) oraz wyszukiwaniu nowych adresów URL do dalszego przetwarzania. Wykryte linki są następnie zwracane jako nowe żądania (`Request`), co pozwala pająkowi na rekurencyjne przeszukiwanie witryny.

### Uruchamianie pająka

Aby uruchomić pająka, przejdź do katalogu projektu `tutorial` i użyj polecenia:

```bash
scrapy crawl quotes
```

Spowoduje to uruchomienie pająka `quotes`. 

Zrzut terminala:

```
...
2025-06-01 10:45:25 [scrapy.core.engine] INFO: Spider opened
2025-06-01 10:45:25 [scrapy.extensions.logstats] INFO: Crawled 0 pages (at 0 pages/min), scraped 0 items (at 0 items/min)
2025-06-01 10:45:25 [scrapy.extensions.telnet] INFO: Telnet console listening on 127.0.0.1:6023
2025-06-01 10:45:25 [scrapy.core.engine] DEBUG: Crawled (404) <GET https://quotes.toscrape.com/robots.txt> (referer: None)
2025-06-01 10:45:25 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://quotes.toscrape.com/page/1/> (referer: None)
2025-06-01 10:45:26 [quotes] DEBUG: Saved file quotes-1.html
2025-06-01 10:45:26 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://quotes.toscrape.com/page/2/> (referer: None)
2025-06-01 10:45:26 [quotes] DEBUG: Saved file quotes-2.html
2025-06-01 10:45:26 [scrapy.core.engine] INFO: Closing spider (finished)
...
```

W katalogu projektu powinny pojawić się pliki `quotes-1.html` i `quotes-2.html`, zawierające pobrane strony z witryny.

### Skrót metody `start()`

Metodę `start()` z kodu powyżej można uprościć zastępując ją przez atrybut klasy `start_urls`. Atrybut ten jest listą startowych adresów URL do odwiedzenia. Lista `start_urls` jest następnie wykorzystywana do automatycznej implementacji metody `start()`.

Kod pająka po zmianie:

```python
from pathlib import Path

import scrapy


class QuotesSpider(scrapy.Spider):
    name = "quotes"
    start_urls = [
        "https://quotes.toscrape.com/page/1/",
        "https://quotes.toscrape.com/page/2/",
    ]

    def parse(self, response):
        page = response.url.split("/")[-2]
        filename = f"quotes-{page}.html"
        Path(filename).write_bytes(response.body)
```

Metoda `parse()` pozostaje bez zmian. Scrapy automatycznie wykorzystuje atrybut `start_urls` i wywołuje metodę `parse()` dla każdego adresu URL z tej listy. Framework oczekuje, że metoda przetwarzająca odpowiedzi będzie miała nazwę `parse`, a lista początkowych adresów – `start_urls`.

### Ekstrakcja danych

Scrapy dostarcza powłokę [Scrapy Shell](https://docs.scrapy.org/en/latest/topics/shell.html#topics-shell) umożliwiającą interaktywne testowanie ekstrakcji danych z witryn. 

Uruchomienie powłoki Scrapy Shell w terminalu:

```bash
scrapy shell "https://quotes.toscrape.com/page/1/"
```

Obiekty dostępne w powłoce:

```
In [1]: shelp()
[s] Available Scrapy objects:
[s]   scrapy     scrapy module (contains scrapy.Request, scrapy.Selector, etc)
[s]   Out        {}
[s]   _oh        {}
[s]   crawler    <scrapy.crawler.Crawler object at 0x7f0a67ec7ec0>
[s]   item       {}
[s]   request    <GET https://quotes.toscrape.com/page/1/>
[s]   response   <200 https://quotes.toscrape.com/page/1/>
[s]   settings   <scrapy.settings.Settings object at 0x7f0a66e0c740>
[s] Useful shortcuts:
[s]   fetch(url[, redirect=True]) Fetch URL and update local objects (by default, redirects are followed)
[s]   fetch(req)                  Fetch a scrapy.Request and update local objects 
[s]   shelp()           Shell help (print this help)
[s]   view(response)    View response in a browser
```

W powłoce można wykonać selekcję danych z odpowiedzi HTTP selektorem XPath lub CSS:

```python
In [3]: response.xpath("//title")
Out[3]: [<Selector query='//title' data='<title>Quotes to Scrape</title>'>]
In [4]: response.css("title")
Out[4]: [<Selector query='descendant-or-self::title' data='<title>Quotes to Scrape</title>'>]
```

Element w postaci tekstowej:

```python
In [9]: response.xpath("//title").get()
Out[9]: '<title>Quotes to Scrape</title>'
In [10]: response.xpath("//title/text()").get()
Out[10]: 'Quotes to Scrape'
```

Wszystkie elementy `small` z atrybutem `class="author"`:

```python
In [16]: response.xpath("//small[attribute::class='author']").getall()
Out[16]: 
['<small class="author" itemprop="author">Albert Einstein</small>',
 '<small class="author" itemprop="author">J.K. Rowling</small>',
 '<small class="author" itemprop="author">Albert Einstein</small>',
 '<small class="author" itemprop="author">Jane Austen</small>',
 '<small class="author" itemprop="author">Marilyn Monroe</small>',
 '<small class="author" itemprop="author">Albert Einstein</small>',
 '<small class="author" itemprop="author">André Gide</small>',
 '<small class="author" itemprop="author">Thomas A. Edison</small>',
 '<small class="author" itemprop="author">Eleanor Roosevelt</small>',
 '<small class="author" itemprop="author">Steve Martin</small>']
```

Wartości tekstowe wszystkich elementów `small` z atrybutem `class="author"`:

```python
In [18]: response.xpath("//small[attribute::class='author']/text()").getall()
Out[18]: 
['Albert Einstein',
 'J.K. Rowling',
 'Albert Einstein',
 'Jane Austen',
 'Marilyn Monroe',
 'Albert Einstein',
 'André Gide',
 'Thomas A. Edison',
 'Eleanor Roosevelt',
 'Steve Martin']
```

### Krótkie wprowadzenie do XPath

[Concise XPath](http://plasmasturm.org/log/xpath101/)

[XPath Tutorial](https://zvon.org/comp/r/tut-XPath_1.html#Pages~List_of_XPaths)

[XPath Language Reference](https://www.w3.org/TR/xpath-31/)

XPath to język zapytań służący do nawigowania i wybierania węzłów w dokumentach XML i HTML. Pozwala precyzyjnie określić, które elementy drzewa dokumentu mają zostać wybrane, korzystając z tzw. kroków, osi i predykatów.

#### Podstawowe zasady działania XPath:
- Każde wyrażenie XPath operuje na zbiorze węzłów (najczęściej zaczynając od korzenia dokumentu lub bieżącego węzła)
- Wyrażenie składa się z sekwencji kroków oddzielonych ukośnikiem `/`, gdzie każdy krok wybiera nowy zbiór węzłów na podstawie określonych kryteriów
- Kroki mogą być uzupełnione o predykaty (`[]`), które filtrują wybrane węzły
- Oś (np. `child::`, `descendant::`, `following-sibling::`) określa relację nawigacji względem bieżącego węzła

#### Kluczowe elementy składni:

1. **Ukośnik `/`** – oddziela kolejne kroki nawigacji w drzewie dokumentu
   ```
   /foo/bar
   - Zacznij od korzenia dokumentu
   - Przejdź do elementów 'foo' będących bezpośrednimi dziećmi korzenia
   - Następnie wybierz wszystkie elementy 'bar' będące dziećmi każdego 'foo'
   ```
   Ukośnik na początku (`/`) oznacza rozpoczęcie od korzenia dokumentu. Każdy kolejny ukośnik oddziela kolejne kroki (nie warunki!).

2. **Krok** – pojedynczy etap nawigacji, np. `foo`, `@id`, `descendant::bar`. Każdy krok może zawierać predykaty:
   ```
   /foo[bar]
   - Wybierz elementy 'foo' mające dziecko 'bar'
   ```
   Przejście do kolejnego kroku nie zawsze oznacza zejście głębiej w drzewie – zależy to od użytej osi.

3. **Predykaty `[]`** – filtrują wybrane węzły na podstawie warunków:
   ```
   //div[@class='quote']
   - Wybierz wszystkie elementy 'div' z atrybutem class="quote"
   ```
   Predykaty mogą zawierać własne kroki i warunki.

4. **Oś** – określa relację nawigacji względem bieżącego węzła. Najczęściej używana jest domyślna oś `child::`, ale można stosować inne, np.:
   ```
   /foo/following-sibling::bar
   - Dla każdego 'foo' wybierz wszystkie elementy 'bar' będące jego rodzeństwem po prawej stronie
   ```
   Osi można używać zarówno jako kroku, jak i wewnątrz predykatów.

5. **Przydatne skróty**:
   - `@nazwa` zamiast `attribute::nazwa` (wybór atrybutu)
   - `//foo` zamiast `/descendant-or-self::foo` (wszystkie elementy 'foo' w dokumencie)
   - Domyślną osią jest `child::`, więc nie trzeba jej pisać jawnie

W XPath można łączyć kroki, osie i predykaty, tworząc złożone i precyzyjne zapytania. Każdy predykat może zawierać kolejne kroki i predykaty, co pozwala na bardzo szczegółowe filtrowanie elementów.

### Ekstrakcja cytatów, autorów i tagów

Korzystając z narzędzi deweloperskich przeglądarki widzimy, że każdy cytat w źródle strony ma postać:

```html
<div class="quote" itemscope="" itemtype="http://schema.org/CreativeWork">
        <span class="text" itemprop="text">“The world as we have created it is a process of our thinking. It cannot be changed without changing our thinking.”</span>
        <span>by <small class="author" itemprop="author">Albert Einstein</small>
        <a href="/author/Albert-Einstein">(about)</a>
        </span>
        <div class="tags">
            Tags:
            <meta class="keywords" itemprop="keywords" content="change,deep-thoughts,thinking,world"> 
            <a class="tag" href="/tag/change/page/1/">change</a>
            <a class="tag" href="/tag/deep-thoughts/page/1/">deep-thoughts</a>
            <a class="tag" href="/tag/thinking/page/1/">thinking</a>
            <a class="tag" href="/tag/world/page/1/">world</a>  
        </div>
    </div>
```

W terminalu uruchamiamy powłokę Scrapy Shell:

```bash
scrapy shell "https://quotes.toscrape.com/
```

Wyodrębniamy pierwszy cytat:

```python
In [1]: quote = response.xpath("//div[attribute::class='quote']")[0]
In [2]: quote
Out[1]: <Selector query="//div[attribute::class='quote']" data='<div class="quote" itemscope itemtype...'>
```

Wyciągamy tekst cytatu:

```python
In [3]: quote.xpath("span[attribute::class='text']/text()").get()
Out[3]: '“The world as we have created it is a process of our thinking. It cannot be changed without changing our thinking.”'
```

Autora:

```python
In [4]: quote.xpath("//small[attribute::class='author']/text()").get()
Out[4]: 'Albert Einstein'
```

Tagi:

```python
In [5]: quote.xpath("div[attribute::class='tags']/a/text()").getall()
Out[5]: 
['change', 'deep-thoughts', 'thinking', 'world']
```

To samo w wersji uproszczonej:

```python
In [6]: quote = response.xpath("//div[@class='quote']")[0]
In [7]: quote.xpath("span[@class='text']/text()").get()
Out[7]: '“The world as we have created it is a process of our thinking. It cannot be changed without changing our thinking.”'
In [8]: quote.xpath("span[@class='author']/text()").get()
Out[8]: 'Albert Einstein'
In [9]: quote.xpath("div[@class='tags']/a/text()").getall()
Out[9]: 
['change', 'deep-thoughts', 'thinking', 'world']
```

### Ekstrakcja danych w pająku

Metodę `parse()` piszemy w postaci funkcji generatora zwracającej słowniki z danymi:

```python
import scrapy


class QuotesSpider(scrapy.Spider):
    name = "quotes"
    start_urls = [
        "https://quotes.toscrape.com/page/1/",
        "https://quotes.toscrape.com/page/2/",
    ]

    def parse(self, response):
        for quote in response.xpath("//div[@class='quote']"):
            yield {
                "text": quote.xpath("span[@class='text']/text()").get(),
                "author": quote.xpath("span/small[@class='author']/text()").get(),
                "tags": quote.xpath("div[@class='tags']/a/text()").getall(),
            }
```

Uruchomienie pająka:

```bash
scrapy crawl quotes
```

Zrzut terminala:

```
...
2025-06-02 21:49:20 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://quotes.toscrape.com/page/1/> (referer: None)
2025-06-02 21:49:20 [scrapy.core.scraper] DEBUG: Scraped from <200 https://quotes.toscrape.com/page/1/>
{'text': '“The world as we have created it is a process of our thinking. It cannot be changed without changing our thinking.”', 'author': 'Albert Einstein', 'tags': ['change', 'deep-thoughts', 'thinking', 'world']}
2025-06-02 21:49:20 [scrapy.core.scraper] DEBUG: Scraped from <200 https://quotes.toscrape.com/page/1/>
{'text': '“It is our choices, Harry, that show what we truly are, far more than our abilities.”', 'author': 'J.K. Rowling', 'tags': ['abilities', 'choices']}
...
```



### Zapisywanie danych do pliku

Najprostszym sposobem na zapisanie danych z pająka do pliku jest użycie podczas uruchamiania opcji:
- `-o` - określa nazwę pliku, do którego mają być **dopisane** dane
- `-O` - określa nazwę pliku, do którego mają być zapisane dane, **nadpisując** jego zawartość o ile taki plik już istnieje

```bash
scrapy crawl quotes -O quotes.json
```

Spowoduje to zapisanie danych w formacie JSON do pliku `quotes.json`. Można również użyć innych formatów, takich jak CSV lub XML, zmieniając rozszerzenie pliku:

```bash
scrapy crawl quotes -O quotes.xml
```

Dopisywanie nowych wartości do istniejącego pliku JSON może prowadzić do niepoprawnego formatu. Z tego powodu lepiej jest korzystać z formatu JSON Lines (JSONL):

```bash
scrapy crawl quotes -o quotes.jsonl
```

### Podążanie za linkami

Strona [quotes.toscrape.com](https://quotes.toscrape.com) zawiera na spodzie linki do kolejnych stron z cytatami w elementach postaci:

```html
<ul class="pager">
            <li class="next">
                <a href="/page/2/">Next <span aria-hidden="true">→</span></a>
            </li>         
        </ul>
```

Wyodrębnienie w Scrapy Shell:

```python
In [5]: response.xpath("//ul[@class='pager']/li[@class='next']/a/@href").get()
Out[5]: '/page/2/'
```

Aby pająk mógł podążać za tymi linkami, należy zmodyfikować metodę `parse()` tak, aby rekurencyjnie generowała nowe żądania dla kolejnych stron:

```python
import scrapy


class QuotesSpider(scrapy.Spider):
    name = "quotes"
    start_urls = [
        "https://quotes.toscrape.com/page/1/"
    ]

    def parse(self, response):
        for quote in response.xpath("//div[@class='quote']"):
            yield {
                "text": quote.xpath("span[@class='text']/text()").get(),
                "author": quote.xpath("span/small[@class='author']/text()").get(),
                "tags": quote.xpath("div[@class='tags']/a/text()").getall(),
            }
        
        next_page = response.xpath("//ul[@class='pager']/li[@class='next']/a/@href").get()
        if next_page is not None:
            next_page = response.urljoin(next_page)  # Tworzy pełny URL
            self.log(f"Przechodzę do następnej strony: {next_page}")
            yield scrapy.Request(url=next_page, callback=self.parse)
```

Pierwsze wywołanie iteratora zwróci dane z pierwszej strony. Wywołanie kolejne spowoduje wstawienie żądania `Request` do harmonogramu przetwarzania. `callback` tego żądania to ponownie metoda `parse()`, co pozwala na rekurencyjne przetwarzanie kolejnych stron.

### Podążanie za linkami

Strona [quotes.toscrape.com](https://quotes.toscrape.com) zawiera na dole każdej strony linki nawigacyjne umożliwiające przejście do kolejnych stron z cytatami:



```html
<ul class="pager">
            <li class="next">
                <a href="/page/2/">Next <span aria-hidden="true">→</span></a>
            </li>         
        </ul>
```



Wyodrębnienie odnośnika do następnej strony w Scrapy Shell:



```
In [5]: response.xpath("//ul[@class='pager']/li[@class='next']/a/@href").get()
Out[5]: '/page/2/'
```

Aby pająk mógł przetwarzać kolejne strony, należy zmodyfikować metodę `parse()` tak, aby oprócz ekstrakcji danych generowała również żądania do kolejnych stron:

```python
import scrapy


class QuotesSpider(scrapy.Spider):
    name = "quotes"
    start_urls = [
        "https://quotes.toscrape.com/page/1/"
    ]

    def parse(self, response):
        for quote in response.xpath("//div[@class='quote']"):
            yield {
                "text": quote.xpath("span[@class='text']/text()").get(),
                "author": quote.xpath("span/small[@class='author']/text()").get(),
                "tags": quote.xpath("div[@class='tags']/a/text()").getall(),
            }
        
        next_page = response.xpath("//ul[@class='pager']/li[@class='next']/a/@href").get()
        if next_page is not None:
            next_page = response.urljoin(next_page)  # Tworzy pełny URL
            yield scrapy.Request(url=next_page, callback=self.parse)
```



W tym kodzie występują dwa rodzaje wyrażeń `yield`:
1. Pierwsze generuje słowniki z danymi zebranymi z aktualnej strony (cytatami)
2. Drugie generuje nowe żądanie HTTP do kolejnej strony (o ile istnieje)

Mechanizm działa rekurencyjnie - dla każdej nowej strony ponownie wywoływana jest ta sama metoda `parse()`, która ponownie ekstrahuje dane i szuka linku do następnej strony. Proces trwa aż do napotkania ostatniej strony, która nie zawiera odnośnika "Next".

### Skrót do żądania Request

W powyższym kodzie używamy metody `response.urljoin(next_page)`, aby utworzyć pełny URL do następnej strony. Można to uprościć, korzystając z funkcji `response.follow()`:

```python

```python
import scrapy


class QuotesSpider(scrapy.Spider):
    name = "quotes"
    start_urls = [
        "https://quotes.toscrape.com/page/1/"
    ]

    def parse(self, response):
        for quote in response.xpath("//div[@class='quote']"):
            yield {
                "text": quote.xpath("span[@class='text']/text()").get(),
                "author": quote.xpath("span/small[@class='author']/text()").get(),
                "tags": quote.xpath("div[@class='tags']/a/text()").getall(),
            }
        
        next_page = response.xpath("//ul[@class='pager']/li[@class='next']/a/@href").get()
        if next_page is not None:
            yield response.follow(next_page, callback=self.parse)
```

### Zbieranie danych o autorach

Pająk z dwiema metodami parsującymi:
- `parse()` – zbiera cytaty ze strony oraz wyszukuje linki do stron autorów
- `parse_author()` – odwiedza stronę każdego autora i zbiera szczegółowe dane o nim

```python
import scrapy

class AuthorSpider(scrapy.Spider):
    name = "author"
    start_urls = [
        "https://quotes.toscrape.com"
    ]

    def parse(self, response):
        # Znajdź wszystkie linki do stron autorów na bieżącej stronie
        author_links = response.xpath("//small[@class='author']/following-sibling::a/@href")
        # Dla każdego linku do autora utwórz żądanie i przekaż do metody parse_author
        yield from response.follow_all(author_links, callback=self.parse_author)

        # Sprawdź, czy istnieje link do następnej strony z cytatami
        next_page = response.xpath("//ul[@class='pager']/li[@class='next']/a/@href").get()
        if next_page is not None:
            # Jeśli tak, przejdź do kolejnej strony i powtórz proces
            yield response.follow(next_page, callback=self.parse)

    def parse_author(self, response):
        # Wyodrębnij dane o autorze ze strony autora
        yield {
            "author": response.xpath("//h3[@class='author-title']/text()").get(),
            "born": response.xpath("//span[@class='author-born-date']/text()").get()
        }
```

W powyższym kodzie:
- W metodzie `parse()` wyszukiwane są wszystkie linki prowadzące do stron autorów na bieżącej stronie z cytatami. Funkcja `yield from response.follow_all(author_links, callback=self.parse_author)` automatycznie tworzy żądania HTTP do wszystkich znalezionych linków i przekazuje je do metody `parse_author`, która przetwarza odpowiedzi z tych stron.
- Następnie, jeśli istnieje link do kolejnej strony z cytatami, pająk przechodzi na tę stronę i ponownie wykonuje ekstrakcję linków do autorów oraz ewentualnie przechodzi dalej.
- Metoda `parse_author()` jest wywoływana dla każdej strony autora i wyodrębnia szczegółowe dane, takie jak imię i nazwisko autora oraz data urodzenia.

Składnia:

```
yield from response.follow_all(author_links, callback=self.parse_author)
```

jest skrótem do iteracji po wszystkich znalezionych linkach i generowania żądań `Request` dla każdego z nich. Każda odpowiedź z tych żądań trafia do metody `parse_author`, gdzie następuje ekstrakcja danych o autorze.